# Scan for TF binding motifs and generate base GRN

This notebook demonstrates how to scan for TF binding motifs. The base GRN will be generated by combining the ATAC-seq peaks and motif information.

Much of this code is from CellOracle tutorial: https://morris-lab.github.io/CellOracle.documentation/notebooks/02_motif_scan/02_atac_peaks_to_TFinfo_with_celloracle_20200801.html

### 0. Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import celloracle as co
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
co.__version__

### 1. Reference genome data preparation
Before starting the analysis, we need to make sure the reference genome data is installed with genomepy. If not, please install the correct reference genome using the instructions below.

genomepy installs reference genome data under home directory. But if you have installed or want to install reference genome to another specific location, please specify the place using genomes_dir argument.

In [ ]:
ref_genome = "mm10"
# genomes_directory = "SET GENOMES DIRECTORY HERE"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome,
                                             genomes_dir=genomes_directory)
print(ref_genome, "installation: ", genome_installation)

if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC", genomes_dir=genomes_directory)
else:
    print(ref_genome, "is installed.")

### 2. Load data
In this notebook, we explain how to make a base GRN.

Using the `processed_peak_file.csv` data file that was output from `6-tss-annotation.ipynb`

In [ ]:
data_dir = "data/base_grn_outputs/E9"
wt_or_ko = "KO"
peaks = pd.read_csv(f"{data_dir}/{wt_or_ko}_processed_peak_file.csv", index_col=0)
peaks.head()

Here, the function below will check peak data format, including chromosome name and peak length.

In [ ]:
peaks = ma.check_peak_format(peaks, ref_genome, genomes_dir=genomes_directory)

You can choose to use either a custom TF binding reference or CellOracle’s default motifs during the motif analysis. If you would like to use our default motifs, you can continue to the next step without loading any additional data. I am using the default!!

### 3. Instantiate TFinfo object and search for TF binding motifs
The motif analysis module has a custom class, TFinfo. The TFinfo object executes the steps below.

1. Converts a peak data into a DNA sequences.
2. Scans the DNA sequences searching for TF binding motifs.
3. Post-processes the motif scan results.
4. Converts data into appropriate format. You can convert data into base-GRN. The base GRN data can be formatted as either a python dictionary or pandas dataframe. This output will be the final base GRN used in the GRN model construction step.

If your reference genome file are installed in non-default location, please speficy the location using genomes_dir.



In [ ]:
# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks,
                ref_genome=ref_genome,
                genomes_dir=genomes_directory)

You can specify the TF binding motif data as follows.

tfi.scan(motifs=motifs)

If you do not specify the motifs or set motifs to None, the default motifs will be loaded automatically.

For mouse and human, “gimme.vertebrate.v5.0.” will be used as the default motifs.

For another species, the species-specific TF binding motif data extracted from CisBP ver2.0 will be used.

NEXT, scan for TF binding motifs. This will take a while!!

In [ ]:
%%time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
tfi.scan(fpr=0.02,
         motifs=None,  # If you enter None, default motifs will be loaded.
         verbose=True)

# Save tfinfo object
tfi.to_hdf5(file_path=f"{data_dir}/{wt_or_ko}_test1.celloracle.tfinfo")

In [ ]:
tfi.scanned_df.head()

### 4. Filtering motifs

In [ ]:
# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

### 5. Get final base GRN

In [ ]:
df = tfi.to_dataframe()
df.head()

### 6. Save results
We will use this information when constructing the GRN models later. Save the results.

In [ ]:
# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet(f"{data_dir}/{wt_or_ko}_base_GRN_dataframe.parquet")